## 简介

归因分析是对比某个指标在 2 个时段的变化. 

为此, 我们需要准备 2 个数据集, 分别记为 df_a 和 df_b. 每个数据集由维度和指标构成, 假设 df_a 内容如下:

| 操作系统 | 付费状态 | 用户占比 | 转化率 |
| --- | --- | --- | --- |
| iOS | 已付费 | 20% | 12% |
| iOS | 未付费 | 30% | 15% |
| Android | 已付费 | 10% | 8% |
| Android | 已付费 | 40% | 17% |

其中的维度: 操作系统, 付费状态. 指标: 用户占比, 转化率. 

对数据集的指标进行某种计算, 可以得到整体指标. 这个计算可以用简单的 Python 函数表达:

```python
def get_metrics(df):
    return sum(df['用户占比']*df['转化率'])
```

于是, 两个时段的整体指标变化就是 $\Delta = \text{get_metric}(df\_b) - \text{get_metric}(df\_a)$

我们的分析目标, 就是解释 $\Delta$, 具体来说:

1. 分析从时段 a 到 b, 每个指标 [用户占比, 转化率] 对 $\Delta$ 分别的贡献.
2. 分析同期各个维度下, 每个指标 [用户占比, 转化率] 分别的贡献. 例如: iOS 已付费用户的用户占比变化, 占 $\Delta$ 的比例.

综上, 归因可将 2 个时段的指标差异, 量化归属到每个维度下的每个指标. 

你可能会问: 为啥不用控制变量法? 具体原因可参见 wiki: Shapley Value.

In [ ]:
# ! pip3 install bytedtqs 

In [6]:
from itertools import combinations
from scipy.special import factorial, comb
import pandas as pd
import numpy as np
import random

import bytedtqs

## 读取 Hive 源数据

- 通过 Python 字符串替换传入 SQL 参数. (如果系统化, 可以做得更友好)
- 运行 2 段 SQL 得到 2 个时段的指标数据集. 

In [34]:
# set starting & ending dates for sql
_dates_1 = {'start_date': '2020-09-12', 'end_date': '2020-09-12'}
_dates_2 = {'start_date': '2020-09-19', 'end_date': '2020-09-19'}
# _dates_3 = {'start_date': '2020-08-21', 'end_date': '2020-08-28'}
# _dates_4 = {'start_date': '2020-08-29', 'end_date': '2020-09-07'}


# set dimensions for sql
_dim = ['channel','role']

In [35]:
sql = """
--------------- SEP ---------------

-- ez device_id 的首次登录记录
with ez_did as (
  select
    device_id
    ,min(create_time) as create_time
  from dm_ez.user_device
  where date = regexp_replace('{end_date}', '-', '')
  group by device_id
),

-- eo 新用户 device_id 画像和行为
eo_did as (
  select
  nu.install_date
  ,nu.device_id
  ,nu.os
  ,case 
    when channel_user_name_modify in ('toutiao_promote','toutiaodsp_new') then '内广' 
    when channel_user_name_modify in ('googleadwords_int','store_google') and os='android' then 'Google-Android' 
    when channel_user_name_modify in ('googleadwords_int','store_google') and os='ios' then 'Google-iOS' 
    when channel_user_name_modify='huawei_id' then '华为' 
    when channel_user_name_modify='guangdiantong' then '广点通' 
    when channel_user_name_modify='AppStore' then 'Apple Store' 
    when channel_user_name_modify='oppo_id' then 'OPPO' 
    when channel_user_name_modify='vivo_id' then 'VIVO' 
    when channel_user_name_modify='xiaomi_id' then '小米' 
    when channel_user_name_modify='fensitong' then '粉丝通' 
    when channel_user_name_modify='baiduxinxiliu' then '百度信息流' 
    when channel_user_name_modify='FacebookAds' then 'Facebook' 
    when channel_user_name_modify='store_tengxun' then '应用宝' else '其他' 
  end as channel
  ,if(ez.create_time is not null and from_unixtime(ez.create_time) < nu.install_time, '已登录EZ', '未登录EZ') as ez_registered
  -- 画像数据可能变, 取 max 
  ,coalesce(max(case nu.role
    when 1 then '上班族'
    when 2 then '自由职业' 
    when 3 then '大学生'
    when 4 then '中小学生'
  end),'unknown') as role
  ,coalesce(max(nu.mile_stone_name), 'unknown') as mile_stone_name
  ,coalesce(max(nu.edu), 'unknown') as edu
  ,coalesce(max(nu.age), 'unknown') as age
  ,coalesce(max(nu.city_level_in_profile), 'unknown') as city_level
  ,coalesce(max(nu.career), 'unknown') as career
  -- 为避免记录重复, 一下也是 max
  ,max(if(gap_days = '0days' and enter_camp = 1, 1, 0)) as d0_enter_camp
  ,max(if(gap_days = '0days' and enter_wechat_group = 1, 1, 0)) as d0_enter_group
  ,max(if(gap_days = '3days' and enter_wechat_group = 1, 1, 0)) as d3_enter_group
  ,max(if(gap_days = '3days' and enter_wechat_group = 1, order_cnt, 0)) as d3_group_cnt
  ,max(if(gap_days = '3days' and enter_wechat_group = 1, order_revenue, 0)) as d3_group_revenue
  ,max(if(gap_days = '6days' and enter_wechat_group = 1, 1, 0)) as d6_enter_group
  ,max(if(gap_days = '6days' and enter_wechat_group = 1, order_cnt, 0)) as d6_group_cnt
  ,max(if(gap_days = '6days' and enter_wechat_group = 1, order_revenue, 0)) as d6_group_revenue
from dm_eo.dwd_eo_newer_revenue_di as nu
left join ez_did as ez on nu.device_id = ez.device_id
where nu.date >= regexp_replace('{start_date}','-','')
  and nu.install_date between '{start_date}' and '{end_date}'
  and nu.gap_days in ('0days','3days','6days')
  and (nu.is_test != 1 or nu.is_test is null)
group by nu.install_date
  ,nu.device_id
  ,nu.os
  ,case 
    when channel_user_name_modify in ('toutiao_promote','toutiaodsp_new') then '内广' 
    when channel_user_name_modify in ('googleadwords_int','store_google') and os='android' then 'Google-Android' 
    when channel_user_name_modify in ('googleadwords_int','store_google') and os='ios' then 'Google-iOS' 
    when channel_user_name_modify='huawei_id' then '华为' 
    when channel_user_name_modify='guangdiantong' then '广点通' 
    when channel_user_name_modify='AppStore' then 'Apple Store' 
    when channel_user_name_modify='oppo_id' then 'OPPO' 
    when channel_user_name_modify='vivo_id' then 'VIVO' 
    when channel_user_name_modify='xiaomi_id' then '小米' 
    when channel_user_name_modify='fensitong' then '粉丝通' 
    when channel_user_name_modify='baiduxinxiliu' then '百度信息流' 
    when channel_user_name_modify='FacebookAds' then 'Facebook' 
    when channel_user_name_modify='store_tengxun' then '应用宝' else '其他' 
  end
  ,if(ez.create_time is not null and from_unixtime(ez.create_time) < nu.install_time, '已登录EZ', '未登录EZ')
),

eo_sub as (
  select
    *
    ,sum(1) over() as dnu_sum
    ,sum(d0_enter_group) over() as d0_enter_group_sum
    ,sum(d3_enter_group) over() as d3_enter_group_sum
    ,sum(d6_enter_group) over() as d6_enter_group_sum
  from eo_did
)

select
  {dim}
  ,dnu_sum
  ,sum(1) as dnu
  ,sum(d0_enter_camp) as d0_enter_camp
  ,sum(d0_enter_group) as d0_enter_group
  ,sum(d3_enter_group) as d3_enter_group
  ,sum(d6_enter_group) as d6_enter_group
  ,1.0*sum(1)/dnu_sum as dnu_prop
  ,1.0*sum(d0_enter_group)/d0_enter_group_sum as d0_enter_group_prop
  ,1.0*sum(d3_enter_group)/d3_enter_group_sum as d3_enter_group_prop
  ,1.0*sum(d6_enter_group)/d6_enter_group_sum as d6_enter_group_prop
  ,1.0*sum(d0_enter_camp)/sum(1) as d0_dnu_to_camp
  ,1.0*sum(d0_enter_group)/sum(1) as d0_dnu_to_group
  ,if(sum(d0_enter_camp) = 0, 0, 1.0*sum(d0_enter_group)/sum(d0_enter_camp)) as d0_camp_to_group
  ,if(sum(d0_enter_camp) = 0, 0, 1.0*sum(d3_enter_group)/sum(d0_enter_camp)) as d3_camp_to_group
  ,if(sum(d0_enter_camp) = 0, 0, 1.0*sum(d6_enter_group)/sum(d0_enter_camp)) as d6_camp_to_group
  ,if(sum(d3_enter_group) = 0, 0, 1.0*sum(d3_group_cnt)/sum(d3_enter_group)) as d3_group_to_order
  ,if(sum(d3_group_cnt) = 0, 0, 1.0*sum(d3_group_revenue)/sum(d3_group_cnt)/100) as d3_arpu
  ,if(sum(d6_enter_group) = 0, 0, 1.0*sum(d6_group_cnt)/sum(d6_enter_group)) as d6_group_to_order
  ,if(sum(d6_group_cnt) = 0, 0, 1.0*sum(d6_group_revenue)/sum(d6_group_cnt)/100) as d6_arpu
from eo_sub
group by {dim}, dnu_sum, d0_enter_group_sum, d3_enter_group_sum, d6_enter_group_sum
"""

In [36]:
# generate sql
_dim_str = ', '.join(_dim)
sql_1 = sql.format(**{**_dates_1, 'dim': _dim_str})
sql_2 = sql.format(**{**_dates_2, 'dim': _dim_str})
# sql_3 = sql.format(**{**_dates_3, 'dim': _dim_str})
# sql_4 = sql.format(**{**_dates_4, 'dim': _dim_str})

# refresh client
# app_id = [替换成自己的 token]
# app_key = [替换成自己的 token]
app_id = 'lFKW9WPzA2tHT7Bv3HuNH2UnIonYG75hnWR6maHVo7YYIXqm'
app_key = 'wTmX8lGFWeFFgROnTzOgb9uIrzrTLeDTtPil0LADDYeOayQo'
user_name = 'wufei.97'

client = bytedtqs.TQSClient(app_id=app_id, app_key=app_key)

注意: 为了分析方便, 我们把 df_a 和 df_b 分别命名成 df_ctl 和 df_trt.

In [37]:
job_1 = client.execute_query(user_name=user_name, query=sql_1, disable_log=True)
job_2 = client.execute_query(user_name=user_name, query=sql_2, disable_log=True)
# job_3 = client.execute_query(user_name=user_name, query=sql_3, disable_log=True)
# job_4 = client.execute_query(user_name=user_name, query=sql_4, disable_log=True)

df_1 = pd.read_csv(job_1.get_result().result_url)
df_2 = pd.read_csv(job_2.get_result().result_url)
# df_3 = pd.read_csv(job_3.get_result().result_url)
# df_4 = pd.read_csv(job_4.get_result().result_url)

[2020-09-20 10:15:10,997] - [INFO] - job submitted, job_id: 129523482
[2020-09-20 10:15:11,020] - [INFO] - job_id: 129523482, engine_type: Hive, status: Created
[2020-09-20 10:15:13,047] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing
[2020-09-20 10:15:15,080] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing
[2020-09-20 10:15:17,103] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing
[2020-09-20 10:15:19,130] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing
[2020-09-20 10:15:21,159] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing
[2020-09-20 10:15:23,187] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing, tracking_urls: http://10.112.18.167:9354/api/v1/applications/application_1594115338351_4865120/jobs/job?id=9482
[2020-09-20 10:15:25,214] - [INFO] - job_id: 129523482, engine_type: Spark, status: Processing, tracking_urls: http://10.112.18.167:9354/api/v1/applications

In [58]:
df_ctl = df_1 # control: df_a
df_trt = df_2 # treatment: df_b

## 数据预处理

如开篇所说, 为了做归因分析, 要定义 `get_metrics` 函数. 

在 Hive 数据集的 columns 中, 选定参与 `get_metrics` 函数运算的指标, 存在 VAR_COLS. 同时给数据集整体指标命名, 存在 METRIC_NAME.

为避免 df_a 和 df_b 的维度不一致, 还需要进行维度补齐. 

In [39]:
# # for UG analysis 2020-09-17
# df_ctl = pd.read_excel('./input/input_eo_dnu_conversion_by_video_stage_1_2020-09-17.xlsx')
# df_trt = pd.read_excel('./input/input_eo_dnu_conversion_by_video_stage_2_2020-09-17.xlsx')

# # meta: DIM_COLS, VAR_COLS, METRIC_NAME
# # note: DIM_COLS + VAR_COLS = all columns of sql; follow the order in sql
# DIM_COLS = ['video']
# VAR_COLS = ['dnu','dnu_proportion','conversion']
# METRIC_NAME = 'conversion'

# # operation function: VAR_COLS => METRIC
# def get_metrics(df):
#     return sum(df['dnu_proportion']*df['conversion'])

In [59]:
# meta: DIM_COLS, VAR_COLS, METRIC_NAME
# note: DIM_COLS + VAR_COLS = all columns of sql; follow the order in sql
DIM_COLS = _dim
VAR_COLS = ['dnu','d0_dnu_to_group']
METRIC_NAME = 'd0_dnu_to_group'

# operation function: VAR_COLS => METRIC
def get_metrics(df):
    return sum(df['dnu']*df['d0_dnu_to_group'])/sum(df['dnu'])

In [51]:
# fill NA
df_ctl[DIM_COLS] = df_ctl[DIM_COLS].astype(str).fillna('_')
df_trt[DIM_COLS] = df_trt[DIM_COLS].astype(str).fillna('_')
df_ctl[VAR_COLS] = df_ctl[VAR_COLS].fillna(0)
df_trt[VAR_COLS] = df_trt[VAR_COLS].fillna(0)

# select required columns
DIM_COLS, VAR_COLS = sorted(DIM_COLS), sorted(VAR_COLS)
df_ctl = df_ctl[DIM_COLS + VAR_COLS]
df_trt = df_trt[DIM_COLS + VAR_COLS]

# dim values 
NEW_DIM_COL = '_dim'
DIM_SEP = '' # keep this rare for split

df_ctl[NEW_DIM_COL] = df_ctl[DIM_COLS].apply(lambda x: '-'.join(x), axis=1)
df_trt[NEW_DIM_COL] = df_trt[DIM_COLS].apply(lambda x: '-'.join(x), axis=1)

# drop old dim cols
df_ctl, df_trt = df_ctl.drop(DIM_COLS, axis=1), df_trt.drop(DIM_COLS, axis=1)

# union of dim values
DIM_VALS = pd.concat([df_ctl[NEW_DIM_COL], df_trt[NEW_DIM_COL]]).unique()

# make sure both dataframes have all rows for dim values
for d in DIM_VALS:
    new_row = dict()
    new_row[NEW_DIM_COL] = d
    for v in VAR_COLS:
        new_row[v] = 0
    if d not in df_ctl[NEW_DIM_COL].values:
        df_ctl = df_ctl.append(new_row, ignore_index=True)
    if d not in df_trt[NEW_DIM_COL].values:
        df_trt = df_trt.append(new_row, ignore_index=True)

/opt/jupyter_deploy/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/jupyter_deploy/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


## 归因分析

原始的 Shapley Value 有 subset 操作, 理论的复杂度是 $O(2^n)$.

$\varphi _{i}(v)=\sum _{S\subseteq N\setminus \{i\}}{\frac {|S|!\;(n-|S|-1)!}{n!}}(v(S\cup \{i\})-v(S))$

为了解决这个问题, 利用抽样, 与 "控制变量法" 做了一点权衡. 损失精度, 用 `random.seed()` 保障可复现.

In [52]:
# players: dim x variable
players = [(i, j) for i in range(len(DIM_VALS)) for j in range(len(VAR_COLS))]
phi = dict()

# sample
N = len(players)
SAMPLE_SIZE = 30
SAMPLE_SIZE = min(SAMPLE_SIZE, factorial(N))
seq_list = list()

random.seed(666)
for _ in range(SAMPLE_SIZE):
    seq = list(range(N))
    random.shuffle(seq)
    seq_list.append(seq)

In [53]:
# reuse the same set of sequences for all players
for seq in seq_list:
    # make of copy of ctl
    df_s = df_ctl.copy()
    # current utility
    v_current = get_metrics(df_s)

    for i in range(N):
        # select player p
        p = players[seq[i]]
        # select dim and variable
        d, v = DIM_VALS[p[0]], VAR_COLS[p[1]]
        # update df_s
        df_s.loc[lambda x: x[NEW_DIM_COL]==d, v] = \
            df_trt.loc[lambda x: x[NEW_DIM_COL]==d, v].values
        # calculate marginal utility
        v_si = get_metrics(df_s)
        phi_i = v_si - v_current
        # update current utility
        v_current = v_si
        
        # add utility for player p
        if p in phi:
            phi[p] += phi_i
        else:
            phi[p] = phi_i

# divided by sample size
phi_avg = {k:1.0*v/SAMPLE_SIZE for k, v in phi.items()}

## 展示结果

这部分有些改进空间, 可以更友好, 改进也很简单. 目前是通过看 Top 因素, 定位主要的影响因素. 

In [54]:
# overall metrics
metrics_ctl, metrics_trt = \
get_metrics(df_ctl), get_metrics(df_trt)
delta_metrics = metrics_trt - metrics_ctl

# standardize
phi_sum = sum(phi.values())
phi_std = {k:1.0*delta_metrics*v/phi_sum for k, v in phi.items()}

# sum of positive and negative contribution
phi_sum_pos, phi_sum_neg = 0, 0

for _, v in phi_std.items():
    if v > 0:
        phi_sum_pos += v
    else:
        phi_sum_neg += v

# save contribution of each player
player_contribution = \
[{'_dim':DIM_VALS[k[0]], 
  '_var':VAR_COLS[k[1]],
  'control': df_ctl.loc[lambda x: x[NEW_DIM_COL]==DIM_VALS[k[0]], VAR_COLS[k[1]]].values[0],
  'treatment': df_trt.loc[lambda x: x[NEW_DIM_COL]==DIM_VALS[k[0]], VAR_COLS[k[1]]].values[0],
  'contribution': v,
  'ratio_contribution_same_sign': v/phi_sum_pos if v > 0 else v/phi_sum_neg
 } 
 for k, v in phi_std.items()]

# select top contribution players according to overall pos/neg change
if delta_metrics > 0:
    player_contribution.sort(key=lambda x: -x['contribution'])
else:
    player_contribution.sort(key=lambda x: x['contribution'])

In [55]:
N_REASONS = 50


def decimal_select(num):
    """
    format decimal places for common metrics
    """
    if num >= 100:
        return int(num)
    elif num >= 1:
        return round(num, 2)
    else:
        return round(num, 4)
    

print(f"整体指标 {METRIC_NAME} ({decimal_select(metrics_ctl)} => {decimal_select(metrics_trt)}), \
变化量 = {decimal_select(delta_metrics)}")
print("-"*30)
print("主要影响因素:\n")

for i in range(N_REASONS):
    if i <= len(player_contribution)-1:
        r = player_contribution[i]
        print(f"""{i+1}. {r['_dim']} 的 {r['_var']} ({decimal_select(r['control'])} => {decimal_select(r['treatment'])}).""",
        f"""贡献: {decimal_select(r['contribution'])}.""",
        f"""占{'正' if r['contribution'] > 0 else '负'}向的: {int(r['ratio_contribution_same_sign']*100)}%.""")

整体指标 d0_dnu_to_group (0.1569 => 0.1304), 变化量 = -0.0265
------------------------------
主要影响因素:

1. 广点通-大学生 的 dnu (575 => 169). 贡献: -0.0176. 占负向的: 33%.
2. 内广-上班族 的 dnu (843 => 571). 贡献: -0.0089. 占负向的: 17%.
3. 广点通-大学生 的 d0_dnu_to_group (0.36 => 0.284). 贡献: -0.0064. 占负向的: 12%.
4. 华为-上班族 的 d0_dnu_to_group (0.2333 => 0.125). 贡献: -0.0035. 占负向的: 6%.
5. 内广-中小学生 的 dnu (148 => 209). 贡献: -0.0021. 占负向的: 4%.
6. 广点通-unknown 的 d0_dnu_to_group (0.1301 => 0.0508). 贡献: -0.0016. 占负向的: 3%.
7. VIVO-unknown 的 d0_dnu_to_group (0.0521 => 0.019). 贡献: -0.0016. 占负向的: 3%.
8. VIVO-自由职业 的 d0_dnu_to_group (0.1132 => 0.0167). 贡献: -0.0013. 占负向的: 2%.
9. 华为-中小学生 的 dnu (149 => 185). 贡献: -0.0012. 占负向的: 2%.
10. 内广-自由职业 的 dnu (236 => 189). 贡献: -0.0011. 占负向的: 2%.
11. 小米-大学生 的 d0_dnu_to_group (0.2286 => 0.1364). 贡献: -0.0006. 占负向的: 1%.
12. 小米-unknown 的 dnu (109 => 129). 贡献: -0.0006. 占负向的: 1%.
13. OPPO-中小学生 的 dnu (99 => 115). 贡献: -0.0005. 占负向的: 1%.
14. VIVO-unknown 的 dnu (192 => 211). 贡献: -0.0005. 占负向的: 0%.
15. 广点通-上班族 的 dnu (64

In [56]:
# contribution summary by var
df_res_var = pd.DataFrame(player_contribution).groupby('_var')['contribution'].sum().reset_index()
df_res_var

,_var,contribution
0,d0_dnu_to_group,-0.003717
1,dnu,-0.022790


In [57]:
# contribution summary by dim
df_res_dim = pd.merge(
    pd.merge(
    pd.DataFrame(player_contribution).groupby('_dim')['contribution'].sum().sort_values().reset_index(),
        df_ctl,
        on='_dim'
    ),
    df_trt,
    on='_dim'
)

df_res_dim

,_dim,contribution,d0_dnu_to_group_x,dnu_x,d0_dnu_to_group_y,dnu_y
0,广点通-大学生,-0.023980,0.360000,575,0.284024,169
1,内广-上班族,-0.006563,0.268090,843,0.281961,571
2,华为-上班族,-0.003308,0.233333,120,0.125000,136
3,内广-中小学生,-0.002113,0.000000,148,0.000000,209
4,VIVO-unknown,-0.002075,0.052083,192,0.018957,211
5,VIVO-自由职业,-0.001417,0.113208,53,0.016667,60
6,华为-中小学生,-0.001249,0.000000,149,0.000000,185
7,广点通-unknown,-0.000930,0.130081,123,0.050847,59
8,小米-unknown,-0.000926,0.027523,109,0.015504,129
9,小米-大学生,-0.000774,0.228571,35,0.136364,22
